In [1]:
import sys

# This to get the peerless target star DataFrame for example purposes
sys.path.append('/u/tdm/repositories/peerless/prediction')
sys.path.append('/u/tdm/repositories/peerless')
from targets import targets

# The action is here. Depends on vespa & isochrones.
from exosyspop.populations import KeplerBinaryPopulation

/u/tdm/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [2]:
pop = KeplerBinaryPopulation(targets, fB=0.4)

In [3]:
# Accessing secondary properties will initialize a secondary simulation,
# calling pop._generate_binaries().  The first time this is called, the
# secondary property regressors get trained.
pop.radius_B

dmag regressor trained, R2=0.999568419596
qR regressor trained, R2=0.99955940295


array([        nan,  0.78764819,         nan, ...,  0.36103053,
               nan,         nan])

In [4]:
# subsequent calls are much faster; e.g.
pop._generate_binaries()
print(pop.radius_B)
%timeit pop._generate_binaries()

[        nan         nan  0.6332953  ...,         nan  0.57872277
         nan]
10 loops, best of 3: 87.9 ms per loop


In [5]:
# If physical accuracy is important, you can also choose to generate binary properties
# directly from the isochrone, but it's a factor of a few slower:
pop._generate_binaries(use_ic=True)
print(pop.radius_B)
%timeit pop._generate_binaries(use_ic=True)

[        nan         nan         nan ...,         nan  0.16342589
         nan]
1 loops, best of 3: 382 ms per loop


In [6]:
# Similarly, accessing orbital properties will generate them
pop.period

array([  6.73278878e+06,   3.39898800e+04,   1.93183235e+03, ...,
         7.86100476e+02,   1.17364899e+02,   1.64388765e+01])

# Synthetic observations

In [7]:
# Now, we can observe and see what we see.  This takes into account
# duty cycle & data span, as well as geometry.
obs = pop.observe()
print(len(obs))
print(obs.columns)
obs.head()

543
Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec'],
      dtype='object')


,index,period,ecc,w,inc,a,aR,b_pri,b_sec,k,...,T14_pri,T14_sec,T23_pri,T23_sec,dataspan,dutycycle,flux_ratio,n_pri,n_sec,phase_sec
0,1161,11.753698,0.781778,4.103566,1.390338,1.577615e+12,28.546547,5.553966,1.213760,0.342173,...,0.000000,0.028998,0.000000,0.000000,1459.789,0.8750,0.008271,0,111,0.151256
1,1687,2.203436,0.165934,5.512670,1.402180,5.529688e+11,8.410328,1.551908,1.230352,0.485898,...,0.000000,0.062407,0.000000,0.000000,1459.789,0.8750,0.020671,0,566,0.576128
2,1982,147.349543,0.825870,5.774248,1.568272,8.797431e+12,155.800852,0.209216,0.089152,0.455771,...,0.409225,0.175878,0.142709,0.064984,1459.789,0.6989,0.018620,7,6,0.943234
3,2031,2.013856,0.099928,5.701840,1.511820,4.788236e+11,6.404177,0.395403,0.354265,0.164808,...,0.116231,0.105481,0.077830,0.071701,1459.789,0.8751,0.001042,631,627,0.553183
4,2503,62.288389,0.148683,4.212638,1.565245,5.061272e+12,72.149742,0.450479,0.346477,0.295699,...,0.379712,0.300137,0.169209,0.147401,1459.789,0.8751,0.003855,23,23,0.454289


In [8]:
# This is pretty fast, even when generating a new population each time:
%timeit pop.observe(new=True)

1 loops, best of 3: 184 ms per loop


In [9]:
# Even faster if we only generate new orbits.
%timeit pop.observe(new_orbits=True)

10 loops, best of 3: 94.8 ms per loop


In [10]:
# So we can predict the expected number of observations pretty easily.
import numpy as np
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True)) for i in range(N)])
n_obs.mean(), n_obs.std()

(532.88, 20.582652890237451)

In [11]:
# Notice that the above does not yet have trapezoidal parameters.  There are two options to generate these.
# Either we can set the fit_trap parameter, as follows:
obs = pop.observe(fit_trap=True)
print(len(obs))
obs.columns

545


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri',
       u'trap_depth_pri', u'trap_slope_pri', u'trap_dur_sec',
       u'trap_depth_sec', u'trap_slope_sec'],
      dtype='object')

In [12]:
# All things considered, this is still pretty fast if we just need to do it a few times:
%timeit pop.observe(fit_trap=True)

1 loops, best of 3: 2.51 s per loop


In [13]:
# However, this is pretty slow if we want to do inference.  To help with this, we can 
# tell it to train & use a regression.  Training only happens once; by default with 10,000 
# synthetic observations.  This takes a minute or so.
obs = pop.observe(regr_trap=True)
print(len(obs))
obs.columns

Depth trained: R2=0.997913681986
Duration trained: R2=0.999125594333
Slope trained: R2=0.998668293268
534


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [14]:
# Subsequent calls are much faster
%timeit pop.observe(regr_trap=True)

10 loops, best of 3: 33 ms per loop


In [15]:
# Even generating a new stellar population & observing it is pretty quick
%timeit pop.observe(regr_trap=True, new=True)

1 loops, best of 3: 200 ms per loop


In [16]:
# Or again, you can just generate new orbits (rather than new binaries & new orbits)
%timeit pop.observe(regr_trap=True, new_orbits=True)

10 loops, best of 3: 114 ms per loop


In [17]:
# Generating the training data used for the trapezoid shape regression above used
# this function, which can be otherwise useful to sample >N random observations 
# from the existing population.  `trap_regr` defaults to `True` here.  
# This function also takes `new` or `new_orbits` keywords.
obs_pop = pop.get_N_observed(N=10000, new_orbits=True)
print(len(obs_pop))
obs_pop.columns

10441


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [18]:
# We can now look, e.g. at the expected number of single/double eclipsing systems:
query = '(n_pri < 3) & (n_sec < 3) & (n_pri==0 | n_sec==0)'
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(6.1299999999999999, 2.4684205476376992)

In [19]:
# Try this again, this time using the empirical eccentricity distribution
# (as opposed to the beta distribution with default params)---eccentricity matters!
pop.ecc_empirical = True
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(10.210000000000001, 3.2780939583849635)